## import libraries

In [ ]:
import cv2, sys, os, random, json, math
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
from numpy import genfromtxt
from sklearn.svm import SVC, NuSVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from stitching.stitch import *

## read in training/testing data

In [ ]:
%cd '/home/uzumochi/eigenjuno/data/train/contrast'

# read in contrasted data and tags
tags = genfromtxt('/home/uzumochi/eigenjuno/data/train/tags.csv', delimiter=',')
vec_size = 256 * 256 * 3;

# ensure dataset class distribution is even
storm_indices = []
no_storm_indices = []

for t in range(350):
    if tags[t] == 1:
        storm_indices.append(t)
    else:
        no_storm_indices.append(t)

number_zeros = int(len(no_storm_indices) / 2)

selected_indices = np.append(random.sample(storm_indices, number_zeros), no_storm_indices)

selected_tags = []

selected_imgs = np.empty((len(selected_indices), vec_size))

curr = 0
for i in selected_indices:
    img = mpimg.imread(str(i + 1) + '.png')
    img = img[:, :, :3]
    img_1d = np.reshape(img, vec_size)
    selected_imgs[curr, :] = img_1d;
    selected_tags = np.append(selected_tags, tags[i])
    curr += 1
    
# tags = np.append(selected_tags, np.full(10, -1)) # int(number_zeros / 4)
# data = np.append(selected_imgs, np.full((10, vec_size), 0), axis = 0)

tags = selected_tags
data = selected_imgs

# set loop parameters
iterations = 1
test_size = 40

## display contrasted dataset

In [ ]:
fig, axes = plt.subplots(int(data.shape[0] / 11), 11, figsize = (15, 100));
for i, ax in enumerate(axes.flat):
    ax.imshow(data[i, :].reshape(256, 256, 3))
    ax.title.set_text(str(int(tags[i])))
    ax.axis('off')
plt.show()

## perform training w/ pca and eigenimaging

In [ ]:
for j in range(iterations):
    # split data into training and test sets
    x_train, x_test, y_train, y_test = train_test_split(data, tags, test_size = test_size)

    # perform original transform again for classification purposes
    pca_x = PCA().fit(x_train)
    x_train_pca = pca_x.transform(x_train)

    # compute eigenfaces
    eigenfaces = pca_x.components_.reshape((x_train.shape[0], 256, 256, 3))

    # apply pca transform to x_test
    x_test_pca = pca_x.transform(x_test)
    
    # fit classifier to data
    classifier_svc = SVC().fit(x_train_pca, y_train)
    classifier_mlp = MLPClassifier(activation = 'logistic').fit(x_train_pca, y_train)
    classifier_nn = KNeighborsClassifier(weights = 'distance').fit(x_train_pca, y_train)
    classifier_nus = NuSVC().fit(x_train_pca, y_train)
    
    # generate predictions
    y_pred_SVC = classifier_svc.predict(x_test_pca)
    y_pred_MLP = classifier_mlp.predict(x_test_pca)
    y_pred_NN = classifier_nn.predict(x_test_pca)
    y_pred_NUS = classifier_nus.predict(x_test_pca)

## print results and analysis

In [ ]:
print(classification_report(y_test, y_pred_SVC))
print(classification_report(y_test, y_pred_MLP))
print(classification_report(y_test, y_pred_NN))
print(classification_report(y_test, y_pred_NUS))

## plot eigenfaces

In [ ]:
eigenfaces = (abs(eigenfaces.astype('float32'))).astype('uint8')
fig, axes = plt.subplots(4, 6, figsize = (15, 10));
for i, ax in enumerate(axes.flat):
    ax.imshow(eigenfaces[i, :]);
    ax.axis('off')
plt.show()

## visualize results w/ images

In [ ]:
fig, axes = plt.subplots(6, 6, figsize = (15, 20));
for i, ax in enumerate(axes.flat):
    ax.imshow(x_test[i - 1, :].reshape(256, 256, 3))
    ax.title.set_text('val: ' + str(int(y_test[i - 1])) + " / pred: " + str(int(y_pred_SVC[i - 1])))
    ax.axis('off')
plt.show()

## contrast single test image

In [ ]:
name = 'test2'
img = cv2.imread('/home/uzumochi/eigenjuno/data/test/' + name + '.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB);
l, a, b = cv2.split(img);
clahe = cv2.createCLAHE(clipLimit = 3.0, tileGridSize = (8, 8));
img_l = clahe.apply(l);
img_l = cv2.merge((img_l, a, b));
final = cv2.cvtColor(img_l, cv2.COLOR_LAB2BGR);
final = cv2.cvtColor(final, cv2.COLOR_BGR2RGB);
img = cv2.resize(final, (2048, 2048), interpolation = cv2.INTER_NEAREST)
plt.imsave('/home/uzumochi/eigenjuno/data/test/' + name + '_contrast.png', img)

## divide-and-conquer pipeline for testing full images

In [ ]:
%cd '/home/uzumochi/eigenjuno/data/test'

# chop up image
test_name = 'test2_contrast'
vec_size = 256 * 256 * 3
img = cv2.imread(test_name + '.png', 3)
b, g, r = cv2.split(img)
img = cv2.merge((r, g, b))
img = (img / 255)
split_img_4 = np.empty((4, vec_size))
split_img_16 = np.empty((16, vec_size))
split_img_64 = np.empty((64, vec_size))
split_img_256 = np.empty((256, vec_size))

In [ ]:
############### 128X128 RESOLUTION BLOCKS ################
start_row, end_row, start_col, end_col = 0, 128, 0, 128
non_zero_blocks = []
index = 0
for i in range(256):
    block = img[start_row : end_row, start_col : end_col]
    if end_col != 2048:
        start_col += 128
        end_col += 128
    else:
        start_col, end_col = 0, 128
        start_row += 128
        end_row += 128
    block = cv2.resize(block, (256, 256), interpolation = cv2.INTER_NEAREST)
    block = np.reshape(block, vec_size)
    split_img_256[i, :] = block
    zero_locs = np.where(block < 0.025)[0]
    if zero_locs.size == 0:
        index += 1
        non_zero_blocks = np.append(non_zero_blocks, i)

# generate predictions for individual blocks
testing_blocks = split_img_256[non_zero_blocks.astype(int), :]
transform_test_256 = pca_x.transform(testing_blocks)
split_pred_256 = classifier_nus.predict(transform_test_256)

# graph results
fig, axes = plt.subplots(16, 16, figsize = (18, 24));
index = 0
for i, ax in enumerate(axes.flat):
    ax.imshow(split_img_256[i, :].reshape(256, 256, 3))
    if i in non_zero_blocks:
        ax.title.set_text(str(float(split_pred_256[index])))
        index += 1
    ax.axis('off')

In [ ]:
############### 256X256 RESOLUTION BLOCKS ################
start_row, end_row, start_col, end_col = 0, 256, 0, 256
non_zero_blocks = []
index = 0
for i in range(64):
    block = img[start_row : end_row, start_col : end_col]
    if end_col != 2048:
        start_col += 256
        end_col += 256
    else:
        start_col, end_col = 0, 256
        start_row += 256
        end_row += 256
    block = np.reshape(block, vec_size)
    split_img_64[i, :] = block
    zero_locs = np.where(block < 0.025)[0]
    if zero_locs.size == 0:
        index += 1
        non_zero_blocks = np.append(non_zero_blocks, i)

# generate predictions for individual blocks
testing_blocks = split_img_64[non_zero_blocks.astype(int), :]
transform_test_64 = pca_x.transform(testing_blocks)
split_pred_64 = classifier_mlp.predict(transform_test_64)

# graph results
fig, axes = plt.subplots(8, 8, figsize = (18, 18));
index = 0
for i, ax in enumerate(axes.flat):
    ax.imshow(split_img_64[i, :].reshape(256, 256, 3))
    if i in non_zero_blocks:
        ax.title.set_text(str(float(split_pred_64[index])))
        index += 1
    ax.axis('off')

In [ ]:
############### 512X512 RESOLUTION BLOCKS ################
start_row, end_row, start_col, end_col = 0, 512, 0, 512
non_zero_blocks = []
index = 0
for i in range(16):
    block = img[start_row : end_row, start_col : end_col]
    if end_col != 2048:
        start_col += 512
        end_col += 512
    else:
        start_col, end_col = 0, 512
        start_row += 512
        end_row += 512
    block = cv2.resize(block, (256, 256))
    block = np.reshape(block, vec_size)
    split_img_16[i, :] = block
    zero_locs = np.where(block < 0.025)[0]
    if zero_locs.size == 0:
        index += 1
        non_zero_blocks = np.append(non_zero_blocks, i)

# generate predictions for individual blocks
testing_blocks = split_img_16[non_zero_blocks.astype(int), :]
transform_test_16 = pca_x.transform(testing_blocks)
split_pred_16 = classifier_nn.predict(transform_test_16)

# graph results
fig, axes = plt.subplots(4, 4, figsize = (18, 18));
index = 0
for i, ax in enumerate(axes.flat):
    ax.imshow(split_img_16[i, :].reshape(256, 256, 3))
    if i in non_zero_blocks:
        ax.title.set_text(str(float(split_pred_16[index])))
        index += 1
    ax.axis('off')    

In [ ]:
############### 1024X1024 RESOLUTION BLOCKS ################
start_row, end_row, start_col, end_col = 0, 1024, 0, 1024
non_zero_blocks = []
index = 0
for i in range(4):
    block = img[start_row : end_row, start_col : end_col]
    if end_col != 2048:
        start_col += 1024
        end_col += 1024
    else:
        start_col, end_col = 0, 1024
        start_row += 1024
        end_row += 1024
    block = cv2.resize(block, (256, 256))
    block = np.reshape(block, vec_size)
    split_img_4[i, :] = block
    zero_locs = np.where(block < 0.025)[0]
    if zero_locs.size == 0:
        index += 1
        non_zero_blocks = np.append(non_zero_blocks, i)

# generate predictions for individual blocks
testing_blocks = split_img_4[non_zero_blocks.astype(int), :]
transform_test_4 = pca_x.transform(testing_blocks)
split_pred_4 = classifier_nn.predict(transform_test_4)

# graph results
fig, axes = plt.subplots(2, 2, figsize = (18, 18));
index = 0
for i, ax in enumerate(axes.flat):
    ax.imshow(split_img_4[i, :].reshape(256, 256, 3))
    if i in non_zero_blocks:
        ax.title.set_text(str(float(split_pred_4[index])))
        index += 1
    ax.axis('off')

## contrast images, resize to 256x256, and save

In [ ]:
%cd '/home/uzumochi/eigenjuno/data/train/cropped'
for i in range(281, 376):
    img = cv2.imread(str(i) + '.png')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB);
    l, a, b = cv2.split(img);
    clahe = cv2.createCLAHE(clipLimit = 3.0, tileGridSize = (8, 8));
    img_l = clahe.apply(l);
    img_l = cv2.merge((img_l, a, b));
    final = cv2.cvtColor(img_l, cv2.COLOR_LAB2BGR);
    final = cv2.cvtColor(final, cv2.COLOR_BGR2RGB);
    img = cv2.resize(final, (256, 256), interpolation = cv2.INTER_NEAREST)
    plt.imsave('/home/uzumochi/eigenjuno/data/train/contrast/' + str(i) + '.png', img)

## pca with uncontrasted data - minimum components

In [ ]:
pca = PCA(n_components = 6).fit(data)
plt.figure(figsize = (15, 5));
plt.plot(pca.explained_variance_ratio_.cumsum());
plt.xlabel('Number of Principal Components');
plt.ylabel('Explained Variance Ratio');
plt.title('Explained Variance Ratio of Principal Components in a Contrasted Dataset: First 5 Components');

## pca with uncontrasted data - all components

In [ ]:
pca = PCA().fit(data)
plt.figure(figsize = (15, 5));
plt.plot(pca.explained_variance_ratio_.cumsum());
plt.xlabel('Number of Principal Components');
plt.ylabel('Explained Variance Ratio');
plt.title('Explained Variance Ratio of Principal Components in a Contrasted Dataset');

## pca scatter plot - first two components

In [ ]:
proj = PCA().fit_transform(data)
plt.scatter(proj[:, 0], proj[:, 1], cmap=plt.cm.get_cmap('rainbow', 2));
plt.xlabel('component 1');
plt.ylabel('component 2');
plt.colorbar();

## useful functions for spice matrices

In [ ]:
# convert rotational matrix to euler angles
def rot_matrix_to_euler(R) :
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
    singular = sy < 1e-6
    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0
    return np.array([x, y, z])

# convert euler angles in radians to degrees
def radians_to_degrees(R):
    x = math.degrees(R[0])
    y = math.degrees(R[1])
    z = math.degrees(R[2])
    return np.array([x, y, z])

## read test image metadata

In [1]:
import json, math
import numpy as np
import spiceypy as spice

path = '/home/uzumochi/eigenjuno/stitching/kernels/'
KERNEL_LIST = [ path+"naif0012.tls", path+"de430.bsp", path+"juno_v12.tf", 
                path+"jup310.bsp", path+"jno_sclkscet_00094.tsc", path+"perijove_9.bsp", 
                path+"pck00010.tpc", path+"perijove_9.bc", path+"juno_junocam_v02.ti" ]
spice.kclear
spice.furnsh(KERNEL_LIST)

JUNO_JUNOCAM = -61500
JUNO_JUNOCAM_BLUE = -61501
JUNO_JUNOCAM_GREEN = -61502
JUNO_JUNOCAM_RED = -61503

with open('/home/uzumochi/eigenjuno/data/test/test2_contrast.json', 'r') as f:
    im_info_dir = json.load(f)
    image = im_info_dir['FILE_NAME']
    image_time = im_info_dir['START_TIME']
    juno_pos = [float(im_info_dir['SUB_SPACECRAFT_LATITUDE']), 
                float(im_info_dir['SUB_SPACECRAFT_LONGITUDE'])]
et = spice.str2et(image_time)

## find lat/long of image centerpoint

In [2]:
# sun_pos = get_sun_jupiter_rel_pos(image_time)
# cam_pos, cam_orient = get_junocam_jupiter_rel_pos_orient(image_time)
# orient_angles = rot_matrix_to_euler(orient)
# orient_angles = radians_to_degrees(orient_angles)

# print('juno subspacecraft lat/long (degrees): ', juno_pos)
# print('sun position relative to jupiter (km): ', sun_pos)
# print('junocam position relative to jupiter (km): ', pos)
# print('junocam orient angles (degrees): ', orient_angles)

# proj_points, ray_mask = project_onto_jupiter_surf(pos, orient)
# proj_points = proj_points[ray_mask, :]

In [10]:
# # Calculate the rotation offset between J2000 and IAU_JUPITER
# rot_matrix = spice.pxform('J2000', 'IAU_JUPITER', et)
# rot_angles = rot_matrix_to_euler(rot_matrix)
# J2000_IAU = radians_to_degrees(rot_angles)

# # Get the rotation of JUNO_SPACECRAFT relative to the IAU_JUPITER frame
# rot_matrix = spice.pxform('IAU_JUPITER', 'JUNO_SPACECRAFT', et)
# rot_angles = rot_matrix_to_euler(rot_matrix)
# IAU_JUNO = radians_to_degrees(rot_angles)

# # Get the rotation of the JUNO_JUNOCAM_CUBE to the SPACECRAFT, based upon data in juno_v12.tf
# rot_matrix = np.matrix([[-0.0059163, -0.0142817, -0.9998805], 
#                         [0.0023828, -0.9998954,  0.0142678], 
#                         [-0.9999797, -0.0022981,  0.0059497]])
# rot_angles = rot_matrix_to_euler(rot_matrix)
# JUNO_CUBE = radians_to_degrees(rot_angles)

# # Get the rotation of JUNO_JUNOCAM to JUNO_JUNOCAM_CUBE, based upon data in juno_v12.tf
# CUBE_CAM = [0.583, -0.469, 0.69]

In [13]:
print(image_time)

# _, frame, bsight, n, bounds = spice.getfov(JUNO_JUNOCAM_BLUE, 4, 32, 32)
# spoint_b, trgepc, srfvec = spice.sincpt(
#     'Ellipsoid', 'JUPITER', et, 'IAU_JUPITER', 'CN+S', 'JUNO_SPACECRAFT', frame, bsight)
# _, long_b, lat_b = spice.reclat(spoint_b)
# lat_b = math.degrees(lat_b)
# long_b = math.degrees(long_b)

_, frame, bsight, n, bounds = spice.getfov(JUNO_JUNOCAM_GREEN, 4, 32, 32)
spoint_g, trgepc, srfvec = spice.sincpt(
    'Ellipsoid', 'JUPITER', et, 'IAU_JUPITER', 'CN+S', 'JUNO_SPACECRAFT', frame, bsight)
_, long_g, lat_g = spice.reclat(spoint_g)
lat_g = math.degrees(lat_g)
long_g = math.degrees(long_g)

# _, frame, bsight, n, bounds = spice.getfov(JUNO_JUNOCAM_RED, 4, 32, 32)
# spoint_r, trgepc, srfvec = spice.sincpt(
#     'Ellipsoid', 'JUPITER', et, 'IAU_JUPITER', 'CN+S', 'JUNO_SPACECRAFT', frame, bsight)
# _, long_r, lat_r = spice.reclat(spoint_r)
# lat_r = math.degrees(lat_r)
# long_r = math.degrees(long_r)

2017-10-24T17:14:31.212


NotFoundError: Spice returns not found for function: sincpt

In [ ]:
# zoom in/out to focus on desired feature


In [ ]:
# track features with centerpoint


In [ ]:
# save in log file w/ coordinates, perijove, and feature id


In [ ]:
# end of notebook